Objective: Can we build a machine learning model that can accurately predict if a borrower will pay off their loan on time or not?

In [1]:
import pandas as pd
loans_2007 = pd.read_csv('LoanStats3a.csv', skiprows=1,low_memory = False)
loans_2007 = loans_2007.drop(['desc', 'url'],axis=1)
half_count = len(loans_2007) / 2
loans_2007 = loans_2007.dropna(thresh=half_count, axis=1)

In [2]:
print(loans_2007.head(1))
print(loans_2007.columns)

        id  member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
0  1077501  1296599.0     5000.0       5000.0           4975.0   36 months   

  int_rate  installment grade sub_grade    ...    last_pymnt_amnt  \
0   10.65%       162.87     B        B2    ...             171.62   

  last_credit_pull_d collections_12_mths_ex_med  policy_code application_type  \
0           Jan-2017                        0.0          1.0       INDIVIDUAL   

  acc_now_delinq chargeoff_within_12_mths delinq_amnt pub_rec_bankruptcies  \
0            0.0                      0.0         0.0                  0.0   

  tax_liens  
0       0.0  

[1 rows x 52 columns]
Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code',
       'addr_state', 'dti', 

After analyzing each column, we can conclude that the following features need to be removed:

* id: randomly generated field by Lending Club for unique identification purposes only
* member_id: also a randomly generated field by Lending Club for unique identification purposes only
* funded_amnt: leaks data from the future (after the loan is already started to be funded)
* funded_amnt_inv: also leaks data from the future (after the loan is already started to be funded)
* grade: contains redundant information as the interest rate column (int_rate)
* sub_grade: also contains redundant information as the interest rate column (int_rate)
* emp_title: requires other data and a lot of processing to potentially be useful
* issue_d: leaks data from the future (after the loan is already completed funded)
* zip_code: redundant with the addr_state column since only the first 3 digits of the 5 digit zip code are visible (which only can be used to identify the state the borrower lives in)
* out_prncp: leaks data from the future, (after the loan already started to be paid off)
* out_prncp_inv: also leaks data from the future, (after the loan already started to be paid off)
* total_pymnt: also leaks data from the future, (after the loan already started to be paid off)
* total_pymnt_inv: also leaks data from the future, (after the loan already started to be paid off)
* total_rec_prncp: also leaks data from the future, (after the loan already started to be paid off)
* total_rec_int: leaks data from the future, (after the loan already started to be paid off),
* total_rec_late_fee: also leaks data from the future, (after the loan already started to be paid off),
* recoveries: also leaks data from the future, (after the loan already started to be paid off),
* collection_recovery_fee: also leaks data from the future, (after the loan already started to be paid off),
* last_pymnt_d: also leaks data from the future, (after the loan already started to be paid off),
* last_pymnt_amnt: also leaks data from the future, (after the loan already started to be paid off).

Recall that Lending Club assigns a grade and a sub-grade based on the borrower's interest rate. While the grade and sub_grade values are categorical, the int_rate column contains continuous values, which are better suited for machine learning. The out_prncp and out_prncp_inv both describe the outstanding principal amount for a loan, which is the remaining amount the borrower still owes. These 2 columns as well as the total_pymnt column describe properties of the loan after it's fully funded and started to be paid off. This information isn't available to an investor before the loan is fully funded and we don't want to include it in our model. All of these columns leak data from the future, meaning that they're describing aspects of the loan after it's already been fully funded and started to be paid off by the borrower.

In [3]:
loans_2007 = loans_2007.drop(['id','member_id','funded_amnt','funded_amnt_inv','grade','sub_grade','emp_title',
                              'issue_d'],axis = 1)
loans_2007 = loans_2007.drop(['zip_code','out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv',
                             'total_rec_prncp'],axis = 1)
loans_2007 = loans_2007.drop(["total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt"], axis=1)
print(loans_2007.iloc[0])
print(loans_2007.shape[1])

loan_amnt                            5000
term                            36 months
int_rate                           10.65%
installment                        162.87
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
addr_state                             AZ
dti                                 27.65
delinq_2yrs                             0
earliest_cr_line                 Jan-1985
inq_last_6mths                          1
open_acc                                3
pub_rec                                 0
revol_bal                           13648
revol_util                          83.7%
total_acc                               9
initial_list_status                     f
last_credit_pull_d               J

In [4]:
print(loans_2007['loan_status'].value_counts())

Fully Paid                                             34108
Charged Off                                             5662
Does not meet the credit policy. Status:Fully Paid      1988
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        10
Current                                                    3
In Grace Period                                            1
Late (16-30 days)                                          1
Default                                                    1
Name: loan_status, dtype: int64


From the investor's perspective, we're interested in trying to predict which loans will be paid off on time and which ones won't be. Only the Fully Paid and Charged Off values describe the final outcome of the loan. The other values describe loans that are still on going and where the jury is still out on if the borrower will pay back the loan on time or not. While the Default status resembles the Charged Off status, in Lending Club's eyes, loans that are charged off have essentially no chance of being repaid while default ones have a small chance. <br/>
Since we're interesting in being able to predict which of these 2 values a loan will fall under, we can treat the problem as a binary classification one. Let's remove all the loans that don't contain either Fully Paid and Charged Off as the loan's status and then transform the Fully Paid values to 1 for the positive case and the Charged Off values to 0 for the negative case. 

Lastly, one thing we need to keep in mind is the class imbalance between the positive and negative cases. While there are 33,136 loans that have been fully paid off, there are only 5,634 that were charged off. This class imbalance is a common problem in binary classification and during training, the model ends up having a strong bias towards predicting the class with more observations in the training set and will rarely predict the class with less observations. The stronger the imbalance, the more biased the model becomes.

In [5]:
vals = ['Fully Paid','Charged Off']
loans_2007 = loans_2007[loans_2007['loan_status'].isin(vals)]
mapping = {'loan_status':{'Fully Paid':1,'Charged Off': 0}}
loans_2007 = loans_2007.replace(mapping)

### Removing Single Value Columns

In [6]:
drop_columns = []
for col in loans_2007.columns:
    col_series = loans_2007[col].dropna().unique()
    if len(col_series) == 1:
        drop_columns.append(col)
loans_2007 = loans_2007.drop(drop_columns, axis=1)
print(drop_columns)

['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']


In [7]:
loans = loans_2007
null_counts = loans.isnull().sum()
print(null_counts)

loan_amnt                 0
term                      0
int_rate                  0
installment               0
emp_length                0
home_ownership            0
annual_inc                0
verification_status       0
loan_status               0
purpose                   0
title                    10
addr_state                0
dti                       0
delinq_2yrs               0
earliest_cr_line          0
inq_last_6mths            0
open_acc                  0
pub_rec                   0
revol_bal                 0
revol_util               50
total_acc                 0
last_credit_pull_d        2
pub_rec_bankruptcies    697
dtype: int64


### Handling Missing Values

While most of the columns have 0 missing values, 2 columns have 50 or less rows with missing values, and 1 column, pub_rec_bankruptcies, contains 697 rows with missing values. Let's remove columns entirely where more than 1% of the rows for that column contain a null value. In addition, we'll remove the remaining rows containing null values.

In [8]:
loans = loans.drop('pub_rec_bankruptcies',axis = 1)
loans = loans.dropna()
print(loans.dtypes.value_counts())
object_columns_df = loans.select_dtypes(include=['object'])
print(object_columns_df.head(1))

object     11
float64    10
int64       1
dtype: int64
         term int_rate emp_length home_ownership verification_status  \
0   36 months   10.65%  10+ years           RENT            Verified   

       purpose     title addr_state earliest_cr_line revol_util  \
0  credit_card  Computer         AZ         Jan-1985      83.7%   

  last_credit_pull_d  
0           Jan-2017  


### Converting Text Columns

Some of the columns seem like they represent categorical values, but we should confirm by checking the number of unique values in those columns:

* home_ownership: home ownership status, can only be 1 of 4 categorical values according to the data dictionary,
* verification_status: indicates if income was verified by Lending Club,
* emp_length: number of years the borrower was employed upon time of application,
* term: number of payments on the loan, either 36 or 60,
* addr_state: borrower's state of residence,
* purpose: a category provided by the borrower for the loan request,
* title: loan title provided the borrower,

In [9]:
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']
for col in cols:
    print(loans[col].value_counts())

RENT        18874
MORTGAGE    17680
OWN          3055
OTHER          96
NONE            3
Name: home_ownership, dtype: int64
Not Verified       16888
Verified           12825
Source Verified     9995
Name: verification_status, dtype: int64
10+ years    8891
< 1 year     4575
2 years      4389
3 years      4093
4 years      3434
5 years      3279
1 year       3239
6 years      2225
7 years      1770
8 years      1481
9 years      1259
n/a          1073
Name: emp_length, dtype: int64
 36 months    29041
 60 months    10667
Name: term, dtype: int64
CA    7093
NY    3813
FL    2868
TX    2728
NJ    1846
IL    1524
PA    1515
VA    1407
GA    1399
MA    1343
OH    1221
MD    1053
AZ     877
WA     841
CO     791
NC     787
CT     754
MI     722
MO     685
MN     613
NV     497
SC     472
WI     459
AL     450
OR     449
LA     436
KY     327
OK     299
KS     271
UT     259
AR     245
DC     212
RI     199
NM     189
WV     177
HI     173
NH     172
DE     113
MT      85
WY      83
AK      

In [10]:
print(loans['purpose'].value_counts())
print(loans['title'].value_counts())

debt_consolidation    18650
credit_card            5133
other                  3984
home_improvement       2980
major_purchase         2182
small_business         1824
car                    1549
wedding                 947
medical                 693
moving                  581
house                   382
vacation                380
educational             320
renewable_energy        103
Name: purpose, dtype: int64
Debt Consolidation                              2188
Debt Consolidation Loan                         1729
Personal Loan                                    661
Consolidation                                    516
debt consolidation                               508
Home Improvement                                 357
Credit Card Consolidation                        357
Debt consolidation                               334
Small Business Loan                              328
Credit Card Loan                                 319
Personal                                         3

In [11]:
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}
loans = loans.drop(['earliest_cr_line','addr_state','title','last_credit_pull_d'],axis = 1)
loans = loans.replace(mapping_dict)
loans["int_rate"] = loans["int_rate"].str.rstrip("%").astype("float")
loans["revol_util"] = loans["revol_util"].str.rstrip("%").astype("float")

In [12]:
cat_columns = ["home_ownership", "verification_status", "emp_length", "purpose", "term"]
dummy_df = pd.get_dummies(loans[cat_columns])
loans = pd.concat([loans, dummy_df], axis=1)
loans = loans.drop(cat_columns, axis=1)

In [13]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
cols = list(loans.columns)
cols.remove('loan_status')
features = loans[cols]
target = loans['loan_status']
lr.fit(features,target)
predictions = lr.predict(features)

### Cross Validation

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict, KFold
lr = LogisticRegression()
kf = KFold(random_state=1)
predictions = cross_val_predict(lr, features, target, cv=kf)
predictions = pd.Series(predictions)

tn = len(predictions[(predictions == 0) & (loans['loan_status']==0)])
tp = len(predictions[(predictions == 1) & (loans['loan_status']==1)])
fn = len(predictions[(predictions == 0) & (loans['loan_status']==1)])
fp = len(predictions[(predictions == 1) & (loans["loan_status"] == 0)])
fpr = (fp/(fp+tn))
tpr = (tp/(tp+fn))
print(fpr,tpr)

0.998757543485978 0.9992940345923049


### Penalizing The Classifier

In [15]:
lr = LogisticRegression(class_weight = 'balanced')
predictions = cross_val_predict(lr, features, target, cv=kf)
predictions = pd.Series(predictions)

tn = len(predictions[(predictions == 0) & (loans['loan_status']==0)])
tp = len(predictions[(predictions == 1) & (loans['loan_status']==1)])
fn = len(predictions[(predictions == 0) & (loans['loan_status']==1)])
fp = len(predictions[(predictions == 1) & (loans["loan_status"] == 0)])
fpr = (fp/(fp+tn))
tpr = (tp/(tp+fn))
print(fpr,tpr)

0.6167909123180688 0.6203671020120014


### Manual Penalties

In [16]:
penalty = {
    0: 10,
    1: 1
}
lr = LogisticRegression(class_weight=penalty)
predictions = cross_val_predict(lr, features, target, cv=kf)
predictions = pd.Series(predictions)

tn = len(predictions[(predictions == 0) & (loans['loan_status']==0)])
tp = len(predictions[(predictions == 1) & (loans['loan_status']==1)])
fn = len(predictions[(predictions == 0) & (loans['loan_status']==1)])
fp = len(predictions[(predictions == 1) & (loans["loan_status"] == 0)])
fpr = (fp/(fp+tn))
tpr = (tp/(tp+fn))
print(fpr,tpr)

0.20837770678026268 0.20305330038828098


In [17]:
from sklearn.ensemble import RandomForestClassifier
lr = RandomForestClassifier(class_weight='balanced',random_state = 1)
predictions = cross_val_predict(lr, features, target, cv=kf)
predictions = pd.Series(predictions)

tn = len(predictions[(predictions == 0) & (loans['loan_status']==0)])
tp = len(predictions[(predictions == 1) & (loans['loan_status']==1)])
fn = len(predictions[(predictions == 0) & (loans['loan_status']==1)])
fp = len(predictions[(predictions == 1) & (loans["loan_status"] == 0)])
fpr = (fp/(fp+tn))
tpr = (tp/(tp+fn))
print(fpr,tpr)

0.9655662051828185 0.9684080480056477


Unfortunately, using a random forest classifier didn't improve our false positive rate. The model is likely weighting too heavily on the 1 class, and still mostly predicting 1s. We could fix this by applying a harsher penalty for misclassifications of 0s.

Ultimately, our best model had a false positive rate of 7%, and a true positive rate of 20%. For a conservative investor, this means that they make money as long as the interest rate is high enough to offset the losses from 7% of borrowers defaulting, and that the pool of 20% of borrowers is large enough to make enough interest money to offset the losses.

If we had randomly picked loans to fund, borrowers would have defaulted on 14.5% of them, and our model is better than that, although we're excluding more loans than a random strategy would. Given this, there's still quite a bit of room to improve:

* We can tweak the penalties further.
* We can try models other than a random forest and logistic regression.
* We can use some of the columns we discarded to generate better features.
* We can ensemble multiple models to get more accurate predictions.
* We can tune the parameters of the algorithm to achieve higher performance.